In [ ]:
%pip install gymnasium

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 10.9 MB/s eta 0:00:00


In [ ]:
import gymnasium as gym
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import RandomizedSearchCV, train_test_split


In [ ]:
Фичи, которые будет менять агент: Flow Duration, Fwd Pkt Len Mean, Bwd Pkt Len Mean, Tot Fwd Pkts, Tot Bwd Pkts

In [ ]:
class Detector:
    def __init__(self, classifier):
        self.classifier = classifier;

    def predict(self, x):
        y = self.classifier.predict(x);
        return y

    def fit(self, data):
        x = data.drop('Label', axis=1)
        y = data['Label']
        x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)
        self.classifier.fit(x_train, y_train)

In [ ]:
class Enviroment(gym.Env):
    def __init__(self, detector, attack_traffic):
        super().__init__()
        self.detector = detector
        self.start = attack_traffic
        self.state = attack_traffic
        self.space = [{"Name": "Flow Duration", "Action": "+Flow Duration", "Value": 100},
                      {"Name": "Flow Duration", "Action": "-Flow Duration", "Value": -100},
                      {"Name": "Fwd Pkt Len Mean", "Action": "+Fwd Pkt Len Mean", "Value": 0.1},
                      {"Name": "Fwd Pkt Len Mean", "Action": "-Fwd Pkt Len Mean", "Value": -0.1},
                      {"Name": "Bwd Pkt Len Mean", "Action": "+Bwd Pkt Len Mean", "Value": 0.1},
                      {"Name": "Bwd Pkt Len Mean", "Action": "-Bwd Pkt Len Mean", "Value": -0.1},
                      {"Name": "Tot Fwd Pkts", "Action": "+Tot Fwd Pkts", "Value": 0.1},
                      {"Name": "Tot Fwd Pkts", "Action": "-Tot Fwd Pkts", "Value": -0.1},
                      {"Name": "Tot Bwd Pkts", "Action": "+Tot Bwd Pkts", "Value": 0.1},
                      {"Name": "Tot Bwd Pkts", "Action": "-Tot Bwd Pkts", "Value": -0.1}]

    def reset(self):
        self.state = self.start
        return self.state

    def step(self, action_n):
        action = self.space[action_n]
        self.state[action["Name"]] += action["Value"]
        result = self.detector.predict(self.state)
        return (self.state, 1 if result == 0 else 1, False)

In [ ]:
class CEM():
    def __init__(self, state_n, action_n):
      self.state_n = state_n
      self.action_n = action_n
      self.policy = np.ones((self.state_n, self.action_n)) / self.action_n

    def get_action(self, state):
      return int(np.random.choice(np.arange(self.action_n), p=self.policy[state]))

    def update_policy(self, elite_tr):
      pre_policy = np.zeros((state_n, action_n))

      for tr in elite_tr:
        for state, action in zip(tr['states'], tr['actions']):
          pre_policy[state][action] += 1

          for state in range(self.action_n):
            if sum(pre_policy[state]) == 0:
              self.policy[state] = np.ones(self.action_n) / self.action_n
            else:
              self.policy[state] = pre_policy[state] / sum(pre_policy[state])

In [ ]:
class CEMAgent():
    def __init__(self, env, agent_logic):
      self.env = env
      self.agent_logic = agent_logic

    def get_state(self):
      return None

    def get_trajectory(self, tr_len):
      tr = {'states': [], 'actions': [], 'total_reward': 0}

      obs = self.env.reset()
      state = get_state(obs)
      tr['states'].append(state)

      for _ in range(tr_len):
          action = self.agent_logic.get_action(state)
          tr['actions'].append(action)

          obs, reward, done = env.step(action)
          state = get_state(obs)
          tr['total_reward'] += reward

          if done:
              break

          tr['states'].append(state)

      return tr

      def get_elite_trajectories(self, trajectories, q):
          total = [tr['total_reward'] for tr in trajectories]
          quantile = np.quantile(total, q=q)
          return [tr for tr in trajectories if tr['total_reward'] > quantile]

      def fit(self, episode_n, trajectory_n, trajectory_len, q):
          for _ in range(episode_n):
              trajectories = [get_trajectory(trajectory_len) for _ in range(trajectory_n)]

              mean_total = np.mean([trajectory['total_reward'] for trajectory in trajectories])
              print(mean_total)
              elite = get_elite_trajectories(trajectories, q)

              if len(elite) > 0:
                  self.agent_logic.update_policy(elite)

In [ ]:
data = pd.read_csv('data.csv')
forest = RandomForestClassifier()
detector = Detector(forest)
detector.fit(data)

In [ ]:
env = Enviroment(detector, None)
agent_logic = CEM(0, 10)
agent = CEMAgent(env, agent_logic)
agent.fit(70, 200, 20. 0.5)